<a href="https://colab.research.google.com/github/Codedestructor56/Medical-Chatbot/blob/main/Medicalchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=d3e02bacb7e3cf949a2dff9b79bb8d3415aac3884041dc22c4eea0b4c42a6fb9
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.5 MB/

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import getpass
token="2arW0r038zGKqfsS9kt5cz2zDLW_44WosAqiLYaTNULVjAfkK"

from pyngrok import ngrok, conf
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = token

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth


In [4]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.llms import Ollama
import pickle
import subprocess
import platform
import os
import tempfile
import uuid


class Chat:
    MAX_MESSAGES = 15
    def __init__(self):
        self.chat_id = str(uuid.uuid4())
        self.history = []
        self.chain = None
        self.message_count = 0

    def __del__(self):
        print(f"Object {self.chat_id} destroyed")

    def create_embeddings(chunks, embedding_model, storing_path="drive/MyDrive/vectorstore"):
        vectorstore = FAISS.from_documents(chunks, embedding_model)
        vectorstore.save_local(storing_path)
        return vectorstore

    def conversation_chat(self, query):
        try:
            result = self.chain({"question": query, "chat_history": self.history})
            self.history.append((query, result["answer"]))
            self.message_count += 1
            return result["answer"]
        except:
            return "Error 500: Failed to Initialize the model due to system constraints"

    def create_conversational_chain(self, vector_store):
        try:
            #llm = Ollama(model="mistral", temperature=0.5)
            llm = LlamaCpp(
            streaming = True,
            model_path="/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
            temperature=0.75,
            top_p=1,
            verbose=True,
            n_gpu_layers = 64,
            n_ctx=2096
          )
            memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

            self.chain = ConversationalRetrievalChain.from_llm(llm=llm, chain_type='stuff',
                                                           retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
                                                           memory=memory)

        except Exception as e:
            try:
                system_platform = platform.system()

                if system_platform == "Linux" or system_platform == "Darwin":
                    subprocess.run(["ollama", "pull", "mistral"], check=True)
                elif system_platform == "Windows":
                    return "Can't run on native windows"
                else:
                    print("Unsupported operating system.")
                    return
            except Exception as e:
                print(f"Error installing Mistral model: {e}")


    def create_chat(self, file_path):
        text_chunks_file_path = 'drive/MyDrive/text_chunks.pkl'
        if os.path.exists(text_chunks_file_path):
            with open(text_chunks_file_path, 'rb') as text_chunks_file:
                text_chunks = pickle.load(text_chunks_file)
        else:
            text = []
            with open(file_path, 'rb') as file:
                file_extension = os.path.splitext(file.name)[1]

                loader = None
                if file_extension == ".pdf":
                    loader = PyPDFLoader(file_path)
                elif file_extension == ".txt":
                    loader = TextLoader(file_path)

                if loader:
                    text.extend(loader.load())


            text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
            text_chunks = text_splitter.split_documents(text)

            with open(text_chunks_file_path, 'wb') as text_chunks_file:
                pickle.dump(text_chunks, text_chunks_file)

        embeddings_file_path = 'drive/MyDrive/embeddings.pkl'
        if os.path.exists(embeddings_file_path):
            with open(embeddings_file_path, 'rb') as embeddings_file:
                embeddings = pickle.load(embeddings_file)
        else:
            embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
            with open(embeddings_file_path, 'wb') as embeddings_file:
                pickle.dump(embeddings, embeddings_file)
        vector_store_file = "drive/MyDrive/vector_store"
        if os.path.exists(vector_store_file):
            vector_store = FAISS.load_local(vector_store_file, embeddings=embeddings)
        else:
            vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
            vector_store.save_local(vector_store_file)

        self.create_conversational_chain(vector_store)

    def rename_chat(self, new_chat_id):
        self.chat_id = new_chat_id

    def reset_chat(self):
        self.history={}
        self.message_count=0

    def reset_chat_history(self):
        if self.message_count >= self.MAX_MESSAGES:
            self.reset_chat()
            print("Chat reset after reaching the maximum number of messages.")
            exit()

def main():
    chat = Chat()
    chat.create_chat(file_path='drive/MyDrive/data/medquad.txt')

    while True:
        user_input = input("You: ")
        if not user_input or user_input=="exit":
            break
        chat.reset_chat_history()
        output = chat.conversation_chat(user_input)

        print(f"Bot: {output}")


main()

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


You: what's a glaucoma?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Bot:  Glaucoma refers to a group of eye disorders that affect the optic nerves, which are responsible for transmitting visual signals from the eyes to the brain. These disorders can cause progressive vision loss and blindness if left untreated. There are two main types of glaucoma: open-angle glaucoma and closed-angle glaucoma. Open-angle glaucoma is characterized by an increase in pressure within the eye, while closed-angle glaucoma occurs when the iris of the eye narrows, obstructing the drainage of fluid from the eye. Treatment for glaucoma typically involves a combination of medication, lifestyle changes, and surgery if necessary. It is important to regularly monitor your eye pressure and seek medical attention if you experience any symptoms of glaucoma, such as blurry vision or seeing halos around lights.
You: so can I have a glaucoma if my eyes are getting teary?


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Bot:  No, tears are not a symptom of glaucoma. Glaucoma is an eye disease characterized by increased pressure in the eye, which can lead to abnormalities such as clouded vision or loss of peripheral vision. While some people with glaucoma may experience dry eyes as a result of the condition, tears themselves are not a symptom of glaucoma.
You: tell me about ulcers, what signs should I look out for?


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Bot:  An ulcer is an inflammation or sore in the lining of your stomach, intestines or esophagus. Signs to look out for include abdominal pain, heartburn, bloating, nausea, vomiting, and changes in bowel habits such as diarrhea or constipation. If you experience any of these symptoms, it's important to see a doctor right away as they may be caused by an underlying condition, such as a bacterial infection, cancer, or autoimmune disease.
You: damn


Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


Bot:  Glaucoma is caused by abnormalities in the channels of the eye that drain fluid from the eye, leading to increased pressure in the eye and potential vision damage.
You: exit


In [5]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [6]:
app = FastAPI()

In [7]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [8]:
chat_instance = Chat()

class UserInput(BaseModel):
    query: str

@app.post("/chat")
async def chat_endpoint(user_input: UserInput):
    chat_instance.reset_chat_history()
    output = chat_instance.conversation_chat(user_input.query)
    return {"response": output}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://9d00-34-127-41-92.ngrok-free.app


INFO:     Started server process [999]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
